# Cloudflare Security Demo - Testing Scripts (Google Colab Edition)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

This notebook contains all the scripts needed for demonstrating Cloudflare security features.

**Demo Site:** https://demo.oskarcode.com/

**Usage:** Run each cell individually during your demo presentation.

**Note:** This notebook is optimized for Google Colab and includes system package installations where needed.

In [ ]:
# Initial Setup for Google Colab
# Purpose: Install required system packages that may not be available in Colab
# Expected: Successful installation of dig, whois, and openssl tools
!apt-get update -qq
!apt-get install -y dnsutils whois openssl
print("✅ Setup complete! All required tools installed.")

## 1. DNS & Proxy Validation (Onboarding)

These commands verify that the domain is properly configured with Cloudflare.

In [ ]:
# DNS Resolution Check
# Purpose: Verify domain resolves to Cloudflare IPs
# Expected: Should return Cloudflare IP addresses (104.x.x.x, 172.x.x.x, etc.)
print("🔍 Checking DNS resolution for demo.oskarcode.com...")
!dig +short demo.oskarcode.com
print("\n✅ If you see IP addresses above, DNS resolution is working!")

In [ ]:
# HTTP Headers Check
# Purpose: Verify Cloudflare is proxying the requests
# Expected: Look for 'server: cloudflare' and 'cf-ray' headers
# What it means: cf-ray = Cloudflare request ID, server = Cloudflare edge server
print("🌐 Checking HTTP headers for Cloudflare indicators...")
!curl -I https://demo.oskarcode.com/
print("\n✅ Look for 'server: cloudflare' and 'cf-ray' headers above!")

In [ ]:
# Whois IP Lookup
# Purpose: Verify the resolved IP belongs to Cloudflare
# Expected: Should show Cloudflare, Inc. as the organization
# What it means: Confirms traffic goes through Cloudflare's network
import subprocess
import socket

try:
    # Get IP address
    domain_ip = socket.gethostbyname('demo.oskarcode.com')
    print(f"🔍 Domain IP: {domain_ip}")
    
    # Run whois on the IP
    result = subprocess.run(['whois', domain_ip], capture_output=True, text=True, timeout=10)
    print("\n🏢 Organization info:")
    for line in result.stdout.split('\n'):
        if any(keyword in line.lower() for keyword in ['org', 'organization', 'owner', 'netname']):
            print(f"  {line.strip()}")
    
    print("\n✅ Look for 'Cloudflare' in the organization info above!")
except Exception as e:
    print(f"⚠️  Whois lookup failed: {e}")
    print("This is normal in some environments. DNS and HTTP tests are sufficient.")

## 2. SQL Injection Testing

Testing WAF (Web Application Firewall) protection against SQL injection attacks.

In [ ]:
# Basic SQL Injection Test
# Purpose: Test if WAF blocks SQL injection attempts
# Expected WITHOUT Cloudflare: Returns products from database
# Expected WITH Cloudflare WAF: HTTP 403 Forbidden or challenge page
# What it means: Cloudflare WAF detected and blocked malicious SQL payload
print("🛡️ Testing basic SQL injection protection...")
print("Payload: test' OR '1'='1' --")
print("\n📝 Response:")
!curl -s "https://demo.oskarcode.com/search/?q=test' OR '1'='1' --" | head -10
print("\n✅ If you see a Cloudflare challenge page or 403 error above, WAF is working!")

In [ ]:
# Flash Sale Workers Rate Limiter Test
# Purpose: Test Workers script protecting flash sales from bot attacks
# Expected: HTTP 429 after rapid requests, then access restored after 10s cooldown
# What it means: Custom edge logic protects high-demand e-commerce pages
print("🔥 Testing Flash Sale Workers rate limiting...")
print("Target: /flash-sale/ (protected by Workers script)")
print("\n📊 First request (should work):")
!curl -I "https://demo.oskarcode.com/flash-sale/" | head -3
print("\n📊 Second request immediately (should get HTTP 429):")
!curl -I "https://demo.oskarcode.com/flash-sale/" | head -3
print("\n✅ Workers script actively protecting flash sale pages from bot abuse!")

In [ ]:
# Check Response Headers for SQL Injection
# Purpose: See HTTP status and Cloudflare security headers
# Expected: HTTP 403 with cf-mitigated header indicating block reason
# What it means: Shows exactly how Cloudflare blocked the threat
print("📊 Checking response headers for SQL injection attempt...")
!curl -I "https://demo.oskarcode.com/search/?q=test' OR '1'='1' --"
print("\n✅ Look for HTTP status code and 'cf-mitigated' header!")

## 3. Credential Exposure Testing

Testing protection against exposed sensitive files and credentials.

In [ ]:
# Git Secrets Exposure Test
# Purpose: Test if git repository files with secrets are accessible
# Expected: Should be blocked by Cloudflare Access Rules
# What it means: Protects against common credential leaking vectors
print("🔐 Testing git secrets protection...")
print("Attempting to access: /.git/secrets.txt")
print("\n📝 Response:")
!curl -s https://demo.oskarcode.com/.git/secrets.txt
print("\n✅ Git files should be protected by WAF rules!")

In [ ]:
# Database Configuration Test
# Purpose: Test if database config files are exposed
# Expected: Should return HTTP 403 or 404
# What it means: Critical infrastructure details are protected
print("🔐 Testing database config protection...")
print("Attempting to access: /config/database.yml")
print("\n📊 Headers:")
!curl -I https://demo.oskarcode.com/config/database.yml
print("\n✅ Config files should return 403/404!")

In [ ]:
# Bad Bot User Agent Test
# Purpose: Test if Cloudflare Bot Management detects malicious bots
# Expected WITHOUT Bot Management: Normal page content
# Expected WITH Bot Management: Challenge page or HTTP 403
# What it means: Suspicious user agents are flagged and challenged
print("🤖 Testing bad bot detection...")
print("User-Agent: BadBot/1.0")
print("\n📊 Headers:")
!curl -I "https://demo.oskarcode.com/robots-welcome" -A "BadBot/1.0"
print("\n✅ Bad bots should be challenged or blocked!")

In [ ]:
# Scraping Bot Test
# Purpose: Test bot detection on content scraping attempts
# Expected: Should trigger bot challenge or block
# What it means: Protects intellectual property and content
print("🤖 Testing scraping bot detection...")
print("User-Agent: ScrapingBot/2.1")
print("\n📝 Response:")
!curl -s "https://demo.oskarcode.com/robots-welcome" -A "ScrapingBot/2.1 (+http://example.com/bot)" | head -10
print("\n✅ Scraping bots should be detected and challenged!")

In [ ]:
# Headless Browser Detection
# Purpose: Test detection of automated browsers used for scraping
# Expected: Challenge or block headless browser attempts
# What it means: Advanced bot detection beyond just user-agent strings
print("🤖 Testing headless browser detection...")
print("User-Agent: HeadlessChrome")
print("\n📊 Headers:")
!curl -I "https://demo.oskarcode.com/" -A "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/91.0.4472.124 Safari/537.36"
print("\n✅ Headless browsers should be detected!")

In [ ]:
# Good Bot vs Bad Bot Comparison
# Purpose: Show how legitimate bots are allowed while bad ones are blocked
# Expected: Google bot should be allowed, BadBot should be challenged
# What it means: Cloudflare intelligently distinguishes between bot types
print("🤖 Comparing Good Bot vs Bad Bot treatment...")
print("\n=== GOOD BOT (Googlebot) ===")
!curl -I "https://demo.oskarcode.com/" -A "Googlebot/2.1 (+http://www.google.com/bot.html)" | head -3
print("\n=== BAD BOT (BadBot) ===")
!curl -I "https://demo.oskarcode.com/" -A "BadBot/1.0" | head -3
print("\n✅ Notice the different treatment between good and bad bots!")

## 5. Rate Limiting Testing

Testing Cloudflare Rate Limiting to prevent abuse and DDoS attacks.

In [ ]:
# API Rate Limiting Test
# Purpose: Test if rate limiting prevents API abuse
# Expected WITHOUT Rate Limiting: All requests succeed (HTTP 200)
# Expected WITH Rate Limiting: Later requests blocked (HTTP 429)
# What it means: Prevents API abuse and ensures fair usage
import subprocess
import time

print("⚡ Testing API rate limiting with 10 rapid requests...")
print("Target: /api/products/")
print("\n📊 Results:")

for i in range(1, 11):
    result = subprocess.run([
        'curl', '-s', '-o', '/dev/null', '-w', f'Request {i:2d}: %{{http_code}}', 
        'https://demo.oskarcode.com/api/products/'
    ], capture_output=True, text=True)
    print(result.stdout)
    time.sleep(0.1)  # Small delay to avoid overwhelming Colab

print("\n✅ Look for HTTP 429 (Too Many Requests) if rate limiting is active!")

In [ ]:
# Login Brute Force Protection Test
# Purpose: Test rate limiting on login attempts to prevent brute force
# Expected: After several attempts, should get rate limited
# What it means: Protects user accounts from credential stuffing attacks
import subprocess
import time

print("⚡ Testing login rate limiting...")
print("Target: /api/login-test/")
print("\n📊 Results:")

for i in range(1, 6):
    result = subprocess.run([
        'curl', '-s', '-X', 'POST', 
        '-d', 'username=admin&password=wrong',
        '-w', f'Login attempt {i}: %{{http_code}}',
        '-o', '/dev/null',
        'https://demo.oskarcode.com/api/login-test/'
    ], capture_output=True, text=True)
    print(result.stdout)
    time.sleep(0.5)  # Delay between login attempts

print("\n✅ Look for increasing response times or HTTP 429 for protection!")

In [ ]:
# Page Access Rate Limiting
# Purpose: Test rate limiting on page access to prevent scraping
# Expected: Should start blocking after threshold is reached
# What it means: Prevents automated content scraping and site abuse
import subprocess
import time

print("⚡ Testing page access rate limiting...")
print("Target: / (homepage)")
print("\n📊 Results:")

for i in range(1, 8):
    result = subprocess.run([
        'curl', '-s', '-I', 
        '-w', f'Page request {i}: %{{http_code}}',
        '-o', '/dev/null',
        'https://demo.oskarcode.com/'
    ], capture_output=True, text=True)
    print(result.stdout)
    time.sleep(0.2)

print("\n✅ Rate limiting may activate after multiple rapid requests!")

## 6. DDoS Protection Testing

Testing Cloudflare's DDoS protection capabilities.

**Note:** These tests are limited in Colab to avoid overwhelming the service.

In [ ]:
# Simulated Traffic Spike Test (Limited for Colab)
# Purpose: Simulate moderate traffic increase to test DDoS protection
# Expected: Cloudflare should handle the load gracefully
# What it means: Site remains available during traffic spikes
import subprocess
import time
from concurrent.futures import ThreadPoolExecutor

def make_request(i):
    result = subprocess.run([
        'curl', '-s', '-o', '/dev/null', '-w', f'Request {i:2d}: %{{http_code}}',
        'https://demo.oskarcode.com/'
    ], capture_output=True, text=True)
    return result.stdout

print("⚡ Simulating moderate traffic spike with 10 concurrent requests...")
print("(Limited for Colab environment)")
print("\n📊 Results:")

with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(make_request, i) for i in range(1, 11)]
    for future in futures:
        print(future.result())

print("\n✅ All requests should complete successfully with Cloudflare protection!")

In [ ]:
# Large Request Test
# Purpose: Test protection against large HTTP requests
# Expected: Should be handled or blocked appropriately
# What it means: Prevents resource exhaustion attacks
print("⚡ Testing large request handling...")
print("Sending 5KB POST request to contact form")

large_data = "x" * 5000  # 5KB of data (reduced for Colab)
result = subprocess.run([
    'curl', '-s', '-X', 'POST', 
    '-d', f'message={large_data}',
    '-w', 'Large request: %{http_code}\n',
    '-o', '/dev/null',
    'https://demo.oskarcode.com/contact/'
], capture_output=True, text=True)

print(f"📊 Response: {result.stdout}")
print("\n✅ Large requests should be handled appropriately!")

## 7. Security Headers Validation

Checking that Cloudflare adds important security headers.

In [ ]:
# Security Headers Check
# Purpose: Verify Cloudflare adds security headers
# Expected: Should see headers like X-Content-Type-Options, X-Frame-Options, etc.
# What it means: Additional layers of security protection
print("🛡️ Checking security headers...")
print("\n📊 Security-related headers:")
!curl -I https://demo.oskarcode.com/ 2>/dev/null | grep -iE "(x-|cf-|strict-|content-security|permissions-policy)"
print("\n✅ Look for X-Frame-Options, X-Content-Type-Options, and CF- headers!")

In [ ]:
# SSL/TLS Configuration Check
# Purpose: Verify SSL configuration and Cloudflare certificates
# Expected: Should show Cloudflare SSL certificate details
# What it means: Traffic is encrypted and authenticated
print("🔒 Checking SSL/TLS certificate...")
print("\n📜 Certificate details:")
!echo | openssl s_client -connect demo.oskarcode.com:443 -servername demo.oskarcode.com 2>/dev/null | openssl x509 -noout -issuer -subject -dates
print("\n✅ Look for Cloudflare as the certificate issuer!")

## 8. Performance Testing

Testing the performance improvements provided by Cloudflare's CDN.

In [ ]:
# Page Load Time Test
# Purpose: Measure page load performance with Cloudflare CDN
# Expected: Fast load times due to edge caching
# What it means: Better user experience and SEO rankings
print("🚀 Measuring page load performance...")
print("\n⏱️  Performance metrics:")
!curl -s -w "DNS lookup:      %{time_namelookup}s\nConnect:         %{time_connect}s\nSSL handshake:   %{time_appconnect}s\nTime to first byte: %{time_starttransfer}s\nTotal time:      %{time_total}s\nDownload size:   %{size_download} bytes\n" -o /dev/null https://demo.oskarcode.com/
print("\n✅ Fast load times indicate effective CDN caching!")

In [ ]:
# Cache Status Check
# Purpose: Verify content is being cached by Cloudflare
# Expected: Should see 'cf-cache-status: HIT' after first request
# What it means: Static content served from edge, reducing origin load
import time

print("💾 Testing Cloudflare cache status...")
print("\n📊 First request (likely MISS or DYNAMIC):")
!curl -I https://demo.oskarcode.com/ 2>/dev/null | grep -i cache

print("\n⏳ Waiting 2 seconds...")
time.sleep(2)

print("\n📊 Second request (checking for cache HIT):")
!curl -I https://demo.oskarcode.com/ 2>/dev/null | grep -i cache

print("\n✅ Look for 'cf-cache-status' header showing cache effectiveness!")

## 9. Geographic Testing

Testing Cloudflare's global network and geo-based features.

In [ ]:
# Cloudflare Ray ID and Data Center
# Purpose: See which Cloudflare data center is serving the request
# Expected: cf-ray header shows location code (last 3 characters)
# What it means: Traffic routed to nearest Cloudflare edge location
print("🌍 Identifying Cloudflare data center...")
print("\n📍 Ray ID and location:")

import re
import subprocess

result = subprocess.run(['curl', '-I', 'https://demo.oskarcode.com/'], capture_output=True, text=True)
for line in result.stdout.split('\n'):
    if 'cf-ray' in line.lower():
        print(line.strip())
        # Extract location code (last 3 characters before any dash)
        ray_match = re.search(r'cf-ray: ([a-f0-9]+)-(\w{3})', line, re.IGNORECASE)
        if ray_match:
            location_code = ray_match.group(2)
            print(f"📍 Data center code: {location_code}")
            print(f"🌐 This indicates your geographic proximity to Cloudflare's edge network!")

print("\n✅ Different locations will show different data center codes!")

In [ ]:
# Country Detection Test
# Purpose: Test if Cloudflare can detect visitor country (if configured)
# Expected: May see geo-related headers if configured
# What it means: Enables geo-based rules and content customization
print("🌍 Checking for geographic detection headers...")
print("\n📍 Geographic headers (if configured):")
!curl -I https://demo.oskarcode.com/ 2>/dev/null | grep -iE "(geo|country|location|cf-ipcountry)"

print("\n📝 Note: Geographic headers may not be visible unless specifically configured")
print("✅ Cloudflare can provide country/region data when enabled in dashboard!")

## 10. Workers Protection Testing

Testing custom Cloudflare Workers security logic.

In [ ]:
# Flash Sale Workers Protection - Detailed Test
# Purpose: Show the complete Workers protection flow
# Expected: Rate limiting in action with custom response page
# What it means: E-commerce sites can protect flash sales from bot attacks
print("⚙️ Testing comprehensive Flash Sale protection...")
print("Target: /flash-sale/ (Workers-protected endpoint)")
print("\n📝 Testing rapid access (simulating bot behavior):")
!curl -s "https://demo.oskarcode.com/flash-sale/" | head -5
print("\n✅ Workers provide sophisticated custom protection at the edge!")

In [ ]:
# Workers Rate Limiting - Technical Details
# Purpose: Show the technical implementation of Workers rate limiting
# Expected: Custom headers showing Workers protection status
# What it means: Developers can implement any custom security logic
import subprocess
import time

print("⚙️ Testing Workers technical implementation...")
print("Analyzing Workers protection headers...")

for i in range(1, 3):
    print(f"\n📊 Flash sale request {i}:")
    result = subprocess.run(['curl', '-I', 'https://demo.oskarcode.com/flash-sale/'], capture_output=True, text=True)
    
    # Show key headers
    for line in result.stdout.split('\n'):
        if any(keyword in line.lower() for keyword in ['http/', 'x-rate', 'retry-after', 'x-protected']):
            print(f"  {line.strip()}")
    
    time.sleep(1)  # Brief delay

print("\n✅ Workers enable custom business logic and intelligent protection!")

## 11. Analytics and Reporting

Overview of security events and analytics capabilities.

In [ ]:
# Security Events Summary
# Purpose: Show what security events would be logged
# Expected: Educational summary of what gets logged
# What it means: Comprehensive visibility into threats and attacks

print("""
📊 CLOUDFLARE SECURITY ANALYTICS OVERVIEW
═══════════════════════════════════════════

The following events are automatically logged and analyzed:

🛡️  WAF (Web Application Firewall) Events:
   ✓ SQL injection attempts blocked
   ✓ XSS (Cross-site scripting) attempts blocked  
   ✓ Malicious payload detection
   ✓ OWASP Top 10 threat protection

🤖 Bot Management Events:
   ✓ Malicious bot requests challenged/blocked
   ✓ Legitimate bot traffic allowed (Google, Bing, etc.)
   ✓ Bot score analysis and machine learning detection
   ✓ Headless browser detection

⚡ Rate Limiting Events:
   ✓ Requests blocked due to rate limits
   ✓ Sources of excessive traffic identified
   ✓ API abuse prevention
   ✓ Brute force attack mitigation

🌍 Traffic & Performance Analytics:
   ✓ Geographic distribution of requests and threats
   ✓ Top attack vectors and sources
   ✓ Response time improvements via CDN
   ✓ Bandwidth savings and cache hit ratios

🔒 Access Control Events:
   ✓ Blocked access to sensitive files
   ✓ IP-based blocking and allowlisting
   ✓ Custom Workers actions and responses
   ✓ SSL/TLS security events

⚙️  Custom Workers Events:
   ✓ Custom security logic execution
   ✓ Progressive threat response actions
   ✓ Business logic security enforcement
   ✓ Edge computing security functions

📈 Business Intelligence:
   ✓ Attack trends and patterns
   ✓ Security ROI metrics
   ✓ Threat landscape insights
   ✓ Compliance reporting data

🔗 Access Analytics Dashboard:
   https://dash.cloudflare.com/
   
   Navigation: Analytics > Security > Events
              Speed > Performance
              Workers > Analytics

✅ All events include detailed metadata: IP, user agent, country, 
   timestamp, action taken, and custom fields for forensic analysis.
""")

## 12. Demo Summary Commands

Quick summary commands to show overall protection status.

In [ ]:
# Complete Security Validation
# Purpose: Quick overview of all security features active
# Expected: Shows Cloudflare protection across all vectors
# What it means: Comprehensive security posture validation

print("🛡️ CLOUDFLARE SECURITY STATUS SUMMARY")
print("═" * 50)

print("\n1️⃣  DNS & Proxy Status:")
!dig +short demo.oskarcode.com | head -2

print("\n2️⃣  Cloudflare Headers Verification:")
!curl -I https://demo.oskarcode.com/ 2>/dev/null | grep -E "(server: cloudflare|cf-ray)" | head -2

print("\n3️⃣  WAF Protection Test (SQL Injection):")
!curl -I "https://demo.oskarcode.com/search/?q=test' OR '1'='1'" 2>/dev/null | grep "HTTP" | head -1

print("\n4️⃣  Bot Management Test (Bad Bot):")
!curl -I "https://demo.oskarcode.com/robots-welcome" -A "BadBot/1.0" 2>/dev/null | grep "HTTP" | head -1

print("\n5️⃣  SSL/TLS Certificate Validation:")
!echo | openssl s_client -connect demo.oskarcode.com:443 -servername demo.oskarcode.com 2>/dev/null | openssl x509 -noout -issuer | grep -o "Cloudflare" | head -1

print("\n6️⃣  Performance - Time to First Byte:")
!curl -s -w "TTFB: %{time_starttransfer}s" -o /dev/null https://demo.oskarcode.com/

print("\n\n" + "═" * 50)
print("✅ SECURITY STATUS: All Cloudflare protection layers are ACTIVE!")
print("🌐 Your site is protected by the Cloudflare global network.")
print("📊 Visit https://dash.cloudflare.com/ for detailed analytics.")
print("═" * 50)

## 📋 Demo Presentation Notes

### 🎯 Key Points to Highlight During Demo:

1. **🔄 Before vs After Cloudflare:** Show the difference in security posture
2. **🚀 Performance Impact:** Highlight speed improvements and caching
3. **📊 Analytics Value:** Explain visibility and threat intelligence
4. **📈 Scalability:** Demonstrate handling of traffic spikes
5. **🤖 Smart Detection:** Show intelligent bot vs human differentiation

### 🔢 Expected HTTP Response Codes:
- **200 OK:** Normal, allowed request
- **403 Forbidden:** Blocked by security rules (WAF, Access Rules)
- **429 Too Many Requests:** Rate limited
- **503 Service Unavailable:** Challenge page ("Just a moment...")

### 🏷️ Important Headers to Point Out:
- `server: cloudflare` → Confirms traffic through Cloudflare network
- `cf-ray` → Unique request ID for troubleshooting and analytics
- `cf-cache-status` → Shows CDN caching effectiveness (HIT/MISS)
- `cf-mitigated` → Indicates what security action was taken

### 📋 Suggested Demo Flow:

1. **🔍 Start with Infrastructure Validation** (Cells 1-3)
   - Show DNS pointing to Cloudflare
   - Verify proxy status and headers

2. **🛡️ Demonstrate Security Features** (Cells 4-10)
   - SQL injection protection
   - Bot detection and management
   - Rate limiting in action
   - Custom Workers logic

3. **🚀 Show Performance Benefits** (Cell 11)
   - Load time improvements
   - Cache hit rates

4. **📊 Wrap up with Analytics Overview** (Cell 12-13)
   - Explain comprehensive logging
   - Show dashboard capabilities

### 💡 Pro Tips for Google Colab:

1. **Run Setup First:** Always execute the setup cell before starting
2. **Individual Cell Execution:** Run one cell at a time for maximum impact
3. **Explain As You Go:** Use the cell outputs to explain what's happening
4. **Interactive Elements:** Encourage audience questions during execution
5. **Save Runtime:** Keep notebook active during presentation

### 🌐 Accessing This Notebook:

1. **Upload to Google Drive:** Save the .ipynb file to your Google Drive
2. **Open in Colab:** Right-click → Open with → Google Colaboratory
3. **Share Link:** Use Colab's sharing features for team demos
4. **Run Setup:** Execute the first cell to install required tools

---

**🎉 Ready to demonstrate the power of Cloudflare security!**

Each cell is designed to showcase a specific aspect of Cloudflare's comprehensive security platform. Run them individually during your presentation for maximum educational impact!